In [99]:
!pip install torch torchvision tensorflow keras numpy pandas matplotlib

In [100]:
!pip install torch==2.1.0 torchvision

In [ ]:
import torch
import torchvision.models as models

# Charger un modèle pré-entraîné
model = models.resnet18(pretrained=True)
model.eval()
#print(model)

In [ ]:
#Pruning#
import torch
import torch.nn.utils.prune as prune
import torchvision.models as models
import os
import gzip
import shutil
import gc

# Libérer la mémoire GPU
gc.collect()
torch.cuda.empty_cache()
print('Affichage de Pruning :')
# Configuration du périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Fonction utilitaire pour obtenir la taille du modèle en Ko
def get_model_size(model, filename="temp_model.pth"):
    torch.save(model.state_dict(), filename)
    size_kb = os.path.getsize(filename) / 1024  # Taille en Ko
    os.remove(filename)
    return size_kb
# Fonction pour appliquer le pruning aux couches Conv2d et Linear
def apply_pruning(model, amount=0.9):
    for name, module in model.named_modules():
        if isinstance(module, (torch.nn.Conv2d, torch.nn.Linear)):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')  # Supprime le masque après pruning
# Fonction pour compresser un fichier modèle avec GZIP
def compress_model(input_path, output_path):
    with open(input_path, 'rb') as f_in, gzip.open(output_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    return os.path.getsize(output_path) / 1024  # Taille en Ko
# Charger le modèle pré-entraîné
model = models.resnet18(pretrained=True).to(device)
model.eval()
# Taille avant pruning
initial_size = get_model_size(model)
print(f"Taille du modèle **avant pruning** : {initial_size:.2f} KB")
# Appliquer le pruning (90% des poids supprimés)
apply_pruning(model, amount=0.9)
# Sauvegarde du modèle pruné
pruned_model_path = "resnet18_pruned.pth"
torch.save(model.state_dict(), pruned_model_path)
# Taille après pruning (toujours dense)
pruned_size = get_model_size(model)
print(f"Taille du modèle **après pruning** (dense) : {pruned_size:.2f} KB")
# Compression avec gzip
compressed_path = "resnet18_pruned.pth.gz"
compressed_size = compress_model(pruned_model_path, compressed_path)
print(f"Taille du modèle compressé (.gz) : {compressed_size:.2f} KB")

In [ ]:
#Quantization#
import torch
import torch.quantization
from torchvision import models
print('Affichage de Quantization :')

# 1. Charger le modèle pré-entraîné
model_fp32 = models.resnet18(pretrained=True)
model_fp32.eval()

# 2. Fusionner les modules (Conv + BN + ReLU)
model_fp32_fused = torch.quantization.fuse_modules(
    model_fp32,
    [['conv1', 'bn1', 'relu']]
    + [['layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu'],
       ['layer1.0.conv2', 'layer1.0.bn2'],
       ['layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu'],
       ['layer1.1.conv2', 'layer1.1.bn2']]
)

# 3. Préparer pour la quantification
model_fp32_fused.qconfig = torch.quantization.get_default_qconfig('fbgemm')  # pour x86
torch.quantization.prepare(model_fp32_fused, inplace=True)

# 5. Convertir en modèle quantifié
model_int8 = torch.quantization.convert(model_fp32_fused, inplace=True)
model_int8.eval()

# 6. Tester ou sauvegarder
torch.save(model_int8.state_dict(), "resnet18_quantized.pth")
###

#comparaison des tailles #
import os
def get_model_size(model):
    torch.save(model.state_dict(), "temp_model.pth")
    size = os.path.getsize("temp_model.pth") / 1024
    os.remove("temp_model.pth")
    return size

print("Taille du modèle original:", get_model_size(model), "KB")
print("Taille du modèle quantifié:", get_model_size(model_int8), "KB")
##


In [ ]:
#Knowledge Distillation#
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
print('Affichage de Knowledge Distillation :')
# Configuration
device = torch.device("cpu")
# Modèles Teacher et Student
teacher = models.resnet34(pretrained=True).to(device)
teacher.eval()
student = models.resnet18(pretrained=False).to(device)
student.fc = nn.Linear(512, 1000)  # Adapter si nécessaire
student.train()
# Fonction de distillation
def distillation_loss(student_logits, teacher_logits, labels, T=10.0, alpha=0.7):
    soft_teacher = F.softmax(teacher_logits / T, dim=1)
    soft_student = F.log_softmax(student_logits / T, dim=1)
    loss_soft = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (T ** 2)
    loss_hard = F.cross_entropy(student_logits, labels)
    return alpha * loss_soft + (1 - alpha) * loss_hard
# Données factices (batch de 4 images RGB 224x224)
x = torch.randn(4, 3, 224, 224).to(device)
y = torch.randint(0, 1000, (4,)).to(device)
# Optimiseur
optimizer = torch.optim.Adam(student.parameters(), lr=0.001)
# Entraînement (5 epochs)
for epoch in range(10):
    with torch.no_grad():
       teacher_logits = teacher(x)

    student_logits = student(x)
    loss = distillation_loss(student_logits, teacher_logits, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"📚 Epoch {epoch+1}/10 - Loss: {loss.item():.4f}")
